# 常规装饰器（函数实现）

In [ ]:
from typing import Callable
from functools import wraps
def decorator(fn: Callable):
    @wraps(fn)  # 指明下面的函数是用来包裹fn的函数，如果不加这个，则原函数加上@decorator后将直接被wrapper函数替换，simple_plus.__name__会变成wrapper
    def wrapper(*args,**kwargs):
        fn_name = fn.__name__
        print(f'{fn_name} start')
        result = fn(*args,**kwargs)
        print(f'{fn_name} end')
        return result
    return wrapper

@decorator # 本质上就是将simple_plus替换为decorator(simple_plus)
def simple_plus(a,b):
    return a+b
print(simple_plus.__name__) # 确认原函数依然是原函数，因为@wraps的作用
result = simple_plus('1','2')
print(result)

# 本身也带参数的装饰器
外面再包一层带参数的，里面都不用变

In [ ]:
def welcome(name):
    def decorator(fn: Callable):
        @wraps(fn)
        def wrapper(*args,**kwargs):
            print(f'Welcome {name}')
            print(f'{fn.__name__} start')
            result = fn(*args,**kwargs)
            print(f'{fn.__name__} end')
            return result
        return wrapper
    return decorator

@welcome('User')
def simple_sum(*args):
    return sum(args)

print(simple_sum(1,2,3,4))

# 生成器
当一个函数包含yield语句后，就不再是一个普通的函数，而是一个生成器

return只能执行一次，yield可以执行好多次

In [ ]:
def hello():    # 执行时返回一个可迭代的对象
    print('Step 1:')
    yield 1
    print('Step 2:')
    yield 2
    print('Step 3:')
    yield 3
a = hello() # a是可迭代对象
print(next(a))    # 测试一下a的迭代方法，输出Step 1:1。每次调用next时，函数只会执行到下一个yield语句
for i in a: # for循环迭代a的剩余部分（后台实际上也是在执行next方法）分别输出Step 2:2 Step 3:3
    print(i)

相比于在函数中构造出一个大的list并最后return，使用yield可以大大节省资源

## 实战：构造一个生成从0到n-1的每个数的平方值的生成器

In [ ]:
def square(n):
    for i in range(n):
        yield i**2

for i in square(4):
    print(i)

# 不使用生成器的版本
# def square_wo_generator(n):
#     result = []
#     for i in range(n):
#         result.append(i**2)
#     return result
# for i in square_wo_generator(4):
#     print(i)

## 上下文管理器
一个上下文管理器是一个对象，它定义了运行时的上下文，使用with语句来执行
### with语句
```py
with context as ctx：
    # 使用这个上下文对象
# 上下文对象已经被清除了
```

In [ ]:
with open("mydata.txt","w") as instance: # instance指向的不是open的返回值，而是上下文管理器对象的__enter__方法的返回值
    instance.write("Hello")

In [1]:
import time
start = time.perf_counter()
nums = []
for n in range(10000):
    nums.append(n**2)
stop = time.perf_counter()
elaspsed = stop-start
print(elaspsed)

0.0024603330000000367


In [1]:
import time
start = time.perf_counter()
class Timer:
    def __init__(self):
        self.elapsed = 0
    def __enter__(self):
        self.start = time.perf_counter()
        return self
    def __exit__(self,exc_type,exc_val,exc_tb):
        self.stop = time.perf_counter()
        self.elapsed = self.stop-self.start
        return False

with Timer() as timer:  # 这时候会自动执行timer中的__enter__方法，timer指向的是__enter__方法的返回值
    nums = []
    for n in range(10000):
        nums.append(n**2)
    # 执行结束的时候会自动执行__exit__方法
print(timer.elapsed)

0.0011598000000958564


In [ ]:
with open('1.txt') as f:
    pass

上下文管理器一般用在
* 开-关
* 锁-释放
* 启动-停止
* 改变-重置